In [1]:
#!pip install openpyxl for reading excel

In [2]:
import data_cleaning as dc
dc.main()

The first step in our pipeline involves loading the data into a pandas DataFrame. This is accomplished using the pandas library, which is imported at the beginning of the script.

In [3]:
# Step 1: Load the data
import pandas as pd

df = pd.read_csv('./data/MMNames_clean.csv')

In [4]:
# Step 2: Prepare the data
import data_preprocessing as dp
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

df = dp.preprocess_category(df,'SR_Name')
df = dp.preprocess_onehot(df,'name')

y = df['SR_Name'].values 
X = df.drop(columns=['SR_Name']).values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
print(X_train.shape, X_test.shape)


[nltk_data] Downloading package punkt to
[nltk_data]     /Users/zarninwayoo/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/zarninwayoo/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/zarninwayoo/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


(13659, 13003) (5854, 13003)


In [5]:
# Step 3: Build a NN model with TensorFlow
import tensorflow as tf

def create_classification_model(input_shape, num_classes, params={}):
    model = tf.keras.Sequential([
        tf.keras.Input(shape=input_shape),
        tf.keras.layers.Dense(32, activation='relu'),
        tf.keras.layers.Dense(16, activation='relu'),
        tf.keras.layers.Dense(num_classes, activation='softmax')
    ])

    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    
    return model

In [6]:
# Step 4: Create and train the model
import time

start_time = time.time()
model = create_classification_model(input_shape=[X_train.shape[1]],num_classes=len(df['SR_Name'].unique()), )
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test), verbose=0)

training_time = time.time() - start_time
print(training_time)

2025-06-14 21:34:49.309089: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


22.23220920562744


In [7]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 32)                416128    
                                                                 
 dense_1 (Dense)             (None, 16)                528       
                                                                 
 dense_2 (Dense)             (None, 18)                306       
                                                                 
Total params: 416,962
Trainable params: 416,962
Non-trainable params: 0
_________________________________________________________________


In [12]:
print("Class distribution:")
print(pd.Series(y_train).value_counts().sort_index())

Class distribution:
0     1561
1      648
2      571
3      434
4      620
5      174
6      381
7     1580
8     1334
9      401
10     149
11     932
12    2310
13     175
14    1004
15     428
16     372
17     585
Name: count, dtype: int64


In [8]:
# Step 5: Evaluate the model

from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score

y_pred = model.predict(X_test, batch_size=32, verbose=0)
y_pred = y_pred.argmax(axis=1)
report = classification_report(y_test, y_pred, output_dict=True)
report_df = pd.DataFrame(report).round(2).transpose()
report_df.to_csv('./data/cls_report_test.csv', index=False)




In [9]:
print(accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

0.3025281858558251
              precision    recall  f1-score   support

           0       0.45      0.22      0.29       617
           1       0.37      0.12      0.18       273
           2       0.44      0.18      0.26       243
           3       0.93      0.18      0.30       214
           4       0.53      0.17      0.26       258
           5       0.89      0.15      0.25        54
           6       0.53      0.19      0.28       171
           7       0.48      0.29      0.36       664
           8       0.43      0.19      0.27       569
           9       0.52      0.25      0.33       199
          10       0.50      0.08      0.14        85
          11       0.83      0.19      0.31       399
          12       0.19      0.79      0.31       968
          13       0.38      0.11      0.17        82
          14       0.73      0.24      0.36       472
          15       0.50      0.19      0.27       188
          16       0.70      0.29      0.41       152
        

In [10]:
y_pred = model.predict(X_train, batch_size=32, verbose=0)
y_pred = y_pred.argmax(axis=1)
report = classification_report(y_train, y_pred, output_dict=True)
report_df = pd.DataFrame(report).round(2).transpose()
report_df.to_csv('./data/cls_report_train.csv', index=False)

In [11]:
print(accuracy_score(y_train, y_pred))
print(classification_report(y_train, y_pred))

0.8666080972252728
              precision    recall  f1-score   support

           0       0.83      0.87      0.85      1561
           1       0.86      0.77      0.81       648
           2       0.86      0.72      0.79       571
           3       0.98      0.99      0.99       434
           4       0.91      0.92      0.92       620
           5       0.99      0.94      0.96       174
           6       0.88      0.93      0.91       381
           7       0.79      0.87      0.83      1580
           8       0.85      0.80      0.82      1334
           9       0.86      0.86      0.86       401
          10       0.93      0.63      0.75       149
          11       0.97      0.86      0.91       932
          12       0.84      0.87      0.85      2310
          13       0.97      0.95      0.96       175
          14       0.94      0.97      0.95      1004
          15       0.87      0.89      0.88       428
          16       0.94      0.91      0.92       372
        